In [ ]:
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

import Artha.neo
from Artha.neo import Neo
import sqlite3
from tqdm import tqdm

def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = str(row[idx])
    return d

neo = Neo("neo4j://localhost:7687", "neo4j", "part2")
conn = sqlite3.connect("../data/ucheckra_.db")
conn.row_factory = dict_factory
checkra_follows = conn.cursor().execute("select * from following").fetchall()

print("done")

In [ ]:
neo.print_nodes()

In [ ]:
# add checkra follows
for d in tqdm(checkra_follows):
    neo.create_node(d)

In [ ]:
neo.clear_db()

In [ ]:
def user_edges(username):
    conn_temp = sqlite3.connect(f"../data/users/u{username}.db")
    conn_temp.row_factory = dict_factory
    try:
        temp_follows = conn_temp.cursor().execute("select * from following").fetchall()
    except:
        return -1
    else:
        for d in tqdm(temp_follows):
            neo.create_node(d)
            neo.create_relation(username, d["username"])
        return len(temp_follows)

num = user_edges("Steen_Jakobsen")
print("done", "Steen_Jakobsen", num)
# Steen_Jakobsen
# for user in tqdm(checkra_follows):
#     num = user_edges(user["username"])
#     print("done", user["username"], num)